![scholtrack_demo](https://github.com/user-attachments/assets/05360671-9674-498f-87b8-682481f2ad7d)

# ScholTrack Widget Demo

**Author**: [Sergey Prokudin](https://scholar.google.com/citations?user=xSywCzAAAAAJ&hl=en).
[[Project Page](https://github.com/sergeyprokudin/scholtrack)]

This notebook contains a simple widget built on top of the ScholTrack API. The widget allows you to provide a list of URLs from Semantic Scholar with the papers of interest, and fetch the corresponding citations. Please see the project page [README](https://github.com/sergeyprokudin/scholtrack) for detailed information.

In [ ]:
# @title 1. Install ScholTrack
!pip install git+https://github.com/sergeyprokudin/scholtrack.git

## 2. Find the Semantic Scholar Paper IDs for the paper of interest:


ScholTrack uses **Semantic Scholar Paper IDs** to retrieve citations. To find a Paper ID for your paper:

1. Go to [Semantic Scholar](https://www.semanticscholar.org/).
2. Search for the paper.
3. Click on the paper in the results that match your search.
4. The **Paper ID** is the string after the last slash in the URL, e.g., for 3D Gaussian Splatting:

   ```
   https://www.semanticscholar.org/paper/3D-Gaussian-Splatting-for-Real-Time-Radiance-Field-Kerbl-Kopanas/2cc1d857e86d5152ba7fe6a8355c2a0150cc280a
   ```

   The Paper ID is `2cc1d857e86d5152ba7fe6a8355c2a0150cc280a`.

5. Use this Paper ID in ScholTrack to retrieve citations.

In [10]:
# @title 3. Fetch Citations from Semantic Scholar with ScholTrack
import ipywidgets as widgets
from IPython.display import display, clear_output
from scholtrack.api import CitationExplorerAPI
from scholtrack.exporter import CitationExporter
from google.colab import files

# Initialize the API client
api_client = CitationExplorerAPI(api_key=None)

# Function to fetch and export citations
def fetch_and_export_citations(b):
    with output:
        clear_output()
        print("Fetching citations for the provided papers, can take a few minutes for papers with thousands of citations and long lists...")

        # Split the input by commas and strip extra spaces
        paper_ids = [paper_id.strip() for paper_id in paper_ids_input.value.split(",")]
        if not all(paper_ids):
            print("Error: Please ensure all Paper IDs are valid.")
            return

        # Fetch citations for all paper IDs
        citations = api_client.get_citations_for_papers(paper_ids, cites_at_least_n=cites_at_least_n_slider.value)
        n_citations = len(citations)

        # Display first N results based on the display limit
        result_str = f"Found {n_citations} citations in total.\n\n"
        result_str += f"Showing first {display_limit_slider.value} results:\n\n"
        citations_sorted = sorted(citations, key=lambda x: CitationExporter.get_sort_value(x, sort_by_dropdown.value), reverse=True)

        for i, citation in enumerate(citations_sorted[:display_limit_slider.value]):
            citing_paper = citation.get("citingPaper", {})
            title = citing_paper.get("title", "N/A")
            authors = citing_paper.get("authors", [])
            author_list = ", ".join([author["name"] for author in authors])
            citation_count = citing_paper.get("citationCount", 0)
            year = citing_paper.get("year", "N/A")
            abstract = citing_paper.get("abstract", "N/A") if show_abstract_checkbox.value else "Abstract hidden"
            venue = citing_paper.get("venue", "N/A")
            external_ids = citing_paper.get("externalIds", {})
            arxiv_url = f"https://arxiv.org/abs/{external_ids.get('ArXiv', '')}" if external_ids.get("ArXiv") else "N/A"
            semantic_scholar_url = citing_paper.get("url", "N/A")

            print(f"{i+1}. Title: {title}")
            print(f"   Authors: {author_list}")
            print(f"   Citation Count: {citation_count}")
            print(f"   Year: {year}")
            print(f"   Venue: {venue}")
            if show_abstract_checkbox.value:
                print(f"   Abstract: {abstract}")
            print(f"   ArXiv URL: {arxiv_url}")
            print(f"   Semantic Scholar URL: {semantic_scholar_url}\n")

        # Save citations based on the output type and return file download
        output_file = "citations_output." + output_type_dropdown.value
        if output_type_dropdown.value == 'csv':
            CitationExporter.save_to_csv(citations, filename=output_file, sort_by=sort_by_dropdown.value)
        elif output_type_dropdown.value == 'json':
            CitationExporter.save_to_json(citations, filename=output_file)
        elif output_type_dropdown.value == 'txt':
            CitationExporter.save_to_txt(citations, filename=output_file, sort_by=sort_by_dropdown.value, show_abstract=show_abstract_checkbox.value)

        print("Found %d citations, showing top %d. Please see the output file (JSON, CSV, TXT) file for the full list." % (len(citations), display_limit_slider.value))
        files.download(output_file)

# Create interactive widgets
paper_ids_input = widgets.Textarea(
    value='2cc1d857e86d5152ba7fe6a8355c2a0150cc280a, 428b663772dba998f5dc6a24488fff1858a0899f',
    placeholder='Enter Semantic Scholar Paper IDs separated by commas',
    description='Paper IDs:',
    layout=widgets.Layout(width='90%', height='100px')
)

output_type_dropdown = widgets.Dropdown(
    options=['csv', 'json', 'txt'],
    value='csv',
    description='Output Type:',
    layout=widgets.Layout(width='50%')
)

sort_by_dropdown = widgets.Dropdown(
    options=['citations', 'arxiv', 'year'],
    value='citations',
    description='Sort By:',
    layout=widgets.Layout(width='50%')
)

display_limit_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=50,
    step=1,
    description='Display Limit:',
    layout=widgets.Layout(width='60%')
)

cites_at_least_n_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Cites-at-least-n:',
    layout=widgets.Layout(width='60%')
)

show_abstract_checkbox = widgets.Checkbox(
    value=False,
    description='Show Abstracts',
    layout=widgets.Layout(width='30%')
)

run_button = widgets.Button(
    description='Fetch Citations',
    button_style='success',
    layout=widgets.Layout(width='40%', height='50px')
)

# Output area for displaying results
output = widgets.Output()

# When the button is clicked, fetch and export citations
run_button.on_click(fetch_and_export_citations)

# Display a link to the guide for finding Paper IDs
paper_id_guide_link = widgets.HTML(
    '<a href="https://github.com/sergeyprokudin/scholtrack/blob/main/README.md#finding-paper-ids" target="_blank">'
    'How to find Paper IDs?</a>'
)

# Display the widgets
display(
    widgets.VBox([
        paper_ids_input,
        widgets.HBox([output_type_dropdown, sort_by_dropdown]),
        display_limit_slider,
        cites_at_least_n_slider,
        show_abstract_checkbox,
        run_button,
        paper_id_guide_link,
        output
    ])
)
